
"""
More Variables :

Total Races from current race
- Races in the last 7/14/28/60/90/180/365/730 days 
- Races won in the 7/14/28/60/90/180/365/730 days
- Races placed in the 7/14/28/60/90/180/365/730 days


"""


# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:
DEBUG = True

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [6]:

df.ft_raw = pd.read_csv(f'{fileloc}/raw_base_2022-12-03.csv', parse_dates = True)

print(df.ft_raw.shape)
print(df.ft_raw.columns.values.tolist())


(588017, 33)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos']


### A) Create Variable for Race Profile

In [7]:

df_base = df.ft_raw.copy()


In [8]:
df_base.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)

In [9]:

df_base[df_base.DogName.isin(['FOURTH DIMENTION'])].head()


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position,y_margin,y_pos
70,114244064,2,FOURTH DIMENTION,3.0,3,33.4,$4.20,NaN,0.39,0.39,...,400m,Grade 5 T3,Warragul,14 Jan 21,2021-01-14,4.2,8,2,0.39,2
71,114244064,2,FOURTH DIMENTION,8.0,8,33.2,$2.70F,NaN,0.60,0.60,...,400m,Grade 5,Warragul,01 Feb 21,2021-02-01,2.7,7,2,0.60,2
72,114244064,8,FOURTH DIMENTION,3.0,3,33.6,$29.60,NaN,7.60,0.36,...,300m,Grade 5,Healesville,02 Mar 21,2021-03-02,29.6,8,8,7.60,8
73,114244064,3,FOURTH DIMENTION,6.0,6,34.0,$9.20,NaN,2.89,0.46,...,400m,Grade 5,Warragul,11 Mar 21,2021-03-11,9.2,7,3,2.89,3
74,114244064,3,FOURTH DIMENTION,4.0,4,33.9,$35.00,NaN,2.09,1.81,...,400m,Grade 5,Warragul,20 Mar 21,2021-03-20,35.0,8,3,2.09,3


In [10]:

# Creating Grad level variables 
df_base['G5'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 5' in x) | ('Five' in x) else 0 )
df_base['G4'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 4' in x) else 0 )
df_base['G3'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 3' in x) else 0 )
df_base['G2'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 2' in x) else 0 )
df_base['G1'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Grade 1' in x) else 0 )

df_base['Gmix'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Mixed' in x) else 0 )
df_base['Gmas'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Master' in x) else 0 )
df_base['Gmai'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Maiden' in x) else 0 )
df_base['Gh'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Heat' in x) else 0 )
df_base['Gres'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Restricted' in x) else 0 )
df_base['Gnov'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Novice' in x) else 0 )
df_base['Gsp'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Special' in x) | ('S/E' in x) else 0 )
df_base['Gfin'] = df_base['RaceGrade'].apply(lambda x : 1 if ('Final' in x) else 0 )


In [11]:

df_base['RaceGrade_H1'] = df_base['RaceGrade'].apply( lambda x : 'Grade 7' if 'Grade 7' in x \
                                                        else 'Grade 6' if 'Grade 6' in x \
                                                        else 'Grade 5' if 'Grade 5' in x \
                                                        else 'Grade 4' if 'Grade 4' in x \
                                                        else 'Grade 3' if 'Grade 3' in x \
                                                        else 'Grade 2' if 'Grade 2' in x \
                                                        else 'Grade 2' if 'Grade 2' in x \
                                                        else 'Grade 1' if 'Grade 1' in x \
                                                        else 'Mixed' if 'Mixed' in x \
                                                        else 'Master' if 'Master' in x \
                                                        else 'Heat' if 'Heat' in x \
                                                        else 'Restricted' if 'Restricted' in x \
                                                        else 'Special' if 'Special' in x \
                                                        else 'Other' )


In [12]:

df_base['RaceGrade_H1'].value_counts()


Grade 5       233003
Other         133264
Mixed         101860
Grade 6        31790
Heat           29078
Restricted     26836
Grade 7        12113
Grade 4         8767
Special         4828
Grade 1         2654
Grade 3         1789
Master          1420
Grade 2          615
Name: RaceGrade_H1, dtype: int64

In [13]:

grade_fix = df_base.groupby(['RaceGrade_H1','RaceGrade']).agg({'@id_RACE':'nunique'}).reset_index()

grade_fix.to_csv(f'{fileloc}/grade_fix.csv', index = False)


In [14]:

print(df_base.shape)
print(df_base.columns.values.tolist())


(588017, 47)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'G5', 'G4', 'G3', 'G2', 'G1', 'Gmix', 'Gmas', 'Gmai', 'Gh', 'Gres', 'Gnov', 'Gsp', 'Gfin', 'RaceGrade_H1']


In [15]:

print(pd.DataFrame(df_base.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  \
0        0      0        0    0    0       1           0    588017        0   

   Margin2  ...  G1  Gmix  Gmas  Gmai  Gh  Gres  Gnov  Gsp  Gfin  RaceGrade_H1  
0    81676  ...   0     0     0     0   0     0     0    0     0             0  

[1 rows x 47 columns]


In [16]:

#print(df_base1.RaceGrade.value_counts())
#racegrades = df_base1.RaceGrade.value_counts()
#racegrades.to_csv
#racegrades.to_csv(f"{fileloc}/racegrades.csv", index = True)


In [20]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']

# 'TrainerName', 'Track' have been dummified
#cols_to_keep = ['@id_DOG', 'DogName', 'y_runtime', 'RaceId', '@id_RACE', 'position', 'ft_sp', 'x_num_dogs', 'Top3_actual', 'position_random', 'position_fasttrack', 'Winner_fastrack', 'Top3_fastrack', 'Winner_actual', 'Winner_random', 'Top3_random']

cols_to_keep = ['@id_DOG', '@id_RACE','G5', 'G4', 'G3', 'G2', 'G1', 'Gmix', 'Gmas', 'Gmai', 'Gh', 'Gres', 'Gnov', 'Gsp', 'Gfin', 'RaceGrade_H1']

"""
cols_for_drop = ['Place', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin' \
                 , 'Prizemoney', 'TrainerId', 'TrainerName', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date' \
                 , 'Event_Dt','RunTime','ft_sp','position','DogName','RaceId','x_num_dogs']
"""

todaydt = datetime.now().date()
exp_fname = f'dataprep_raceprofile_{todaydt}.csv'

#exp_ds = df_base.drop(columns = cols_for_drop)
exp_ds = df_base[ cols_to_keep ]


In [21]:

print(exp_ds.columns.values.tolist())


['@id_DOG', '@id_RACE', 'G5', 'G4', 'G3', 'G2', 'G1', 'Gmix', 'Gmas', 'Gmai', 'Gh', 'Gres', 'Gnov', 'Gsp', 'Gfin', 'RaceGrade_H1']


In [22]:

exp_ds.head()


,@id_DOG,@id_RACE,G5,G4,G3,G2,G1,Gmix,Gmas,Gmai,Gh,Gres,Gnov,Gsp,Gfin,RaceGrade_H1
0,108390981,626271182,0,0,0,0,0,1,0,0,0,0,0,0,0,Mixed
1,108390981,629797242,1,0,0,0,0,0,0,0,0,0,0,0,0,Grade 5
2,108390981,629797513,1,0,0,0,0,0,0,0,0,0,0,0,0,Grade 5
3,108390981,638810808,0,0,0,0,0,1,0,0,1,0,0,0,0,Mixed
4,108390981,643554706,0,1,0,0,0,0,0,0,0,0,0,0,0,Grade 4


In [23]:
exp_ds.describe()

,@id_DOG,@id_RACE,G5,G4,G3,G2,G1,Gmix,Gmas,Gmai,Gh,Gres,Gnov,Gsp,Gfin
count,5.880170e+05,5.880170e+05,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000
mean,4.619574e+08,7.386968e+08,0.403407,0.014909,0.003042,0.001046,0.004513,0.173226,0.021219,0.152278,0.103643,0.053483,0.037632,0.032438,0.042171
std,1.145309e+08,6.357945e+07,0.490581,0.121191,0.055074,0.032323,0.067031,0.378443,0.144113,0.359290,0.304797,0.224995,0.190304,0.177160,0.200978
min,1.083910e+08,6.238619e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.005005e+08,6.854128e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.569441e+08,7.402821e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.405822e+08,7.922435e+08,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.145360e+09,8.483914e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:

print(pd.DataFrame(exp_ds.isna().sum()).transpose())


   @id_DOG  @id_RACE  G5  G4  G3  G2  G1  Gmix  Gmas  Gmai  Gh  Gres  Gnov  \
0        0         0   0   0   0   0   0     0     0     0   0     0     0   

   Gsp  Gfin  RaceGrade_H1  
0    0     0             0  


In [25]:

exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)


In [26]:

print('THE END')


THE END
